# Table of Contents
* [National Generation Capacities](#National-Generation-Capacities)
	* [Prepare Environment](#Prepare-Environment)
	* [Import raw data from Excel-file](#Import-raw-data-from-Excel-file)
	* [Convert raw data to list](#Convert-raw-data-to-list)
	* [Define technology levels](#Define-technology-levels)
	* [Print results of technology levels](#Print-results-of-technology-levels)
		* [Technology level 1](#Technology-level-1)
		* [Technology level 2](#Technology-level-2)
		* [Technology level 3](#Technology-level-3)
	* [Comparison of different technology levels for all countries](#Comparison-of-different-technology-levels-for-all-countries)
	* [Comparison of different technology levels for a selection](#Comparison-of-different-technology-levels-for-a-selection)
* [Documenting the data package (meta data)](#Documenting-the-data-package-%28meta-data%29)
* [Write results to file](#Write-results-to-file)


# National Generation Capacities

## Prepare Environment 

In [ ]:
import urllib.request
import csv
import pandas as pd
import numpy as np
import posixpath
import urllib.parse
import datetime  
import re
import os.path
import yaml  # http://pyyaml.org/, pip install pyyaml, conda install pyyaml
import json
import subprocess
from bokeh.charts import Scatter, show
from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
import logging
logger = logging.getLogger('notebook')
logger.setLevel('INFO')
nb_root_logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                              datefmt='%d %b %Y %H:%M:%S')
nb_root_logger.handlers[0].setFormatter(formatter)

#create download and output folder if they do not exist
if not os.path.exists('output/'): os.makedirs('output/')
if not os.path.exists('output/datapackage_generation_capacities'): os.makedirs('output/datapackage_generation_capacities')

## Import raw data from Excel-file

In [ ]:
data_file = '20160116_Generation_Capacity_Europe_v01__RET_FK.xlsx'
filepath = 'inputs/'+data_file
data_raw = pd.read_excel(filepath,
                     sheetname='Summary',
                     header=None,
                     na_values = ['-'],
                     skiprows=0)

# Deal with merged cells in Excel:Fill first three rows with information 
data_raw.iloc[0:2] = data_raw.iloc[0:2].fillna(method='ffill', axis=1)

#Set index for rows
data_raw=data_raw.set_index([0])
data_raw.index.name='technology' 

#set multiindex column names
data_raw.columns=pd.MultiIndex.from_arrays(data_raw[:4].values, names=['country','type','year','source']) 

#remove 3 rows which are already used as column names
data_raw = data_raw[pd.notnull(data_raw.index)] 

data_raw

## Convert raw data to list

In [ ]:
# Reshape Dateframe to list
data = pd.DataFrame(data_raw.stack(level=['source','year','type','country']))

# reset index for Dataframe
data=data.reset_index()
data=data.set_index('technology')

# delete entries with missing source
data = data[data['source'].isnull() == False]

data=data.rename(columns={0: 'value'})

data['value'] = pd.to_numeric(data['value'], errors='coerce')

data.head()

## Define technology levels

In [ ]:
data['technology_level'] = '3'

data.loc['Total','technology_level'] = '0'

data.loc['Conventional fuels','technology_level'] = '1'
data.loc['Renewable fuels','technology_level'] = '1'
data.loc['Other fuels','technology_level'] = '1, 2, 3'

data.loc['Fossil fuels','technology_level'] = '2'
data.loc['Nuclear','technology_level'] = '2, 3'
data.loc['Hydro','technology_level'] = '2'
data.loc['Wind','technology_level'] = '2'
data.loc['Solar','technology_level'] = '2'
data.loc['Geothermal','technology_level'] = '2, 3'
data.loc['Tide, wave, and ocean','technology_level'] = '2, 3'
data.loc['Other renewable fuels','technology_level'] = '2'
data['technology_level']

data=data.reset_index()
data.head()


## Print results of technology levels

### Technology level 1

In [ ]:
pivot_capacity_level1 = pd.pivot_table(data[data.technology_level.str.contains('1')],
                               index=('country','year','source'),
                               columns = ('technology'),
                               values='value',
                               aggfunc=sum,
                               margins=False)

pivot_capacity_plot=pivot_capacity_level1.plot(kind='bar',stacked=True, legend=True, figsize=(12, 6))
pivot_capacity_plot.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
pivot_capacity_plot.set_ylim(0,250000)

pivot_capacity_plot
pivot_capacity_level1

### Technology level 2

In [ ]:
pivot_capacity_level2 = pd.pivot_table(data[data.technology_level.str.contains('2')],
                               index=('country','year','source'),
                               columns = ('technology'),
                               values='value',
                               aggfunc=sum,
                               margins=False)

pivot_capacity_plot=pivot_capacity_level2.plot(kind='bar',stacked=True, legend=True, figsize=(12, 6))
pivot_capacity_plot.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
pivot_capacity_plot.set_ylim(0,250000)

pivot_capacity_plot
pivot_capacity_level2

### Technology level 3

In [ ]:
pivot_capacity_level3 = pd.pivot_table(data[data.technology_level.str.contains('3')],
                               index=('country','year','source'),
                               columns = ('technology'),
                               values='value',
                               aggfunc=sum,
                               margins=False)

pivot_capacity_plot=pivot_capacity_level3.plot(kind='bar',stacked=True, legend=True, figsize=(12, 6))
pivot_capacity_plot.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
pivot_capacity_plot.set_ylim(0,250000)

pivot_capacity_plot
pivot_capacity_level3

## Comparison of different technology levels for all countries

In [ ]:
capacity_total_1 = pd.DataFrame(data[data['technology_level'].str.contains('1')].groupby(['source','year','type','country'])['value'].sum())
capacity_total_2 = pd.DataFrame(data[data['technology_level'].str.contains('2')].groupby(['source','year','type','country'])['value'].sum())
capacity_total_3 = pd.DataFrame(data[data['technology_level'].str.contains('3')].groupby(['source','year','type','country'])['value'].sum())

capacity_total_comparison = pd.DataFrame(capacity_total_1)
capacity_total_comparison = pd.merge(capacity_total_1, capacity_total_2,left_index=True,right_index=True,how='left')
capacity_total_comparison = pd.merge(capacity_total_comparison, capacity_total_3,left_index=True,right_index=True,how='left')

capacity_total_comparison = capacity_total_comparison.rename(columns={'value_x': 'technology level 1','value_y': 'technology level 2','value': 'technology level 3'})

capacity_total_comparison = capacity_total_comparison.sortlevel(['country','year'])

capacity_total_pivot_plot = capacity_total_comparison.plot(kind='bar',stacked=False, legend=True, figsize=(12, 6))
capacity_total_pivot_plot.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
#capacity_total_pivot_plot.set_ylim(0,250000)

capacity_total_pivot_plot
capacity_total_comparison

## Comparison of different technology levels for a selection

In [ ]:
capacity_total_comparison = pd.DataFrame(capacity_total_comparison.stack()).reset_index().rename(columns={'level_4': 'technology_level',0: 'value'})

capacity_total_pivot = pd.pivot_table(
                               # select specific country for comparison
#                               capacity_total_comparison[capacity_total_comparison['country']=='BE'],
                               # select specific source for comparison 
                               capacity_total_comparison[capacity_total_comparison['source']=='entsoe'],
                               index=('country','year','source'),
                               columns='technology_level', 
                               values='value',
                               aggfunc=sum,
                               margins=False)

capacity_total_pivot_plot = capacity_total_pivot.plot(kind='bar',stacked=False, legend=True, figsize=(12, 6))
capacity_total_pivot_plot.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
#capacity_total_pivot_plot.set_ylim(0,250000)

capacity_total_pivot_plot
capacity_total_pivot

# Documenting the data package (meta data)

We document the data packages meta data in the specific format JSON as proposed by the Open Knowledge Foundation. See the Frictionless Data project by OKFN (http://data.okfn.org/) and the Data Package specifications (http://dataprotocols.org/data-packages/) for more details.

In order to keep the notebook more readable, we first formulate the metadata in the human-readable YAML format using a multi-line string. We then parse the string into a Python dictionary and save that to disk as a JSON file.

In [ ]:
# Here we define meta data of the resulting data package.
# The meta data follows the specification at:
# http://dataprotocols.org/data-packages/

metadata = """

name: opsd-power-plants-germany
title: List of power plants in Germany.
description: This dataset contains an augmented and corrected power plant list based on the power plant list provided by the BNetzA.
version: 0.0.1
keywords: [power plants,germany]

resources:
    path: power_plants_germany.csv
    format: csv
    mediatype: text/csv
    schema:  # Schema according to: http://dataprotocols.org/json-table-schema/        
        fields:
            - name: id
              description: 
              type: string
            - name: technology
              description: 
              type: string
            - name: source
              description: 
              type: string
            - name: year
              description: 
              type: string
              format:
            - name: type
              description: 
              type: 
              format:
            - name: country
              description: 
              type: 
              format:
            - name: value
              description: 
              type: float
              format:

licenses:
    - url: http://example.com/license/url/here
      name: License Name Here
      version: 1.0
      id: license-id-from-open

sources:
    - name: ,
      web: 
    - name: ,
      web: 

maintainers:
    - name: OPSD-Team,
      email: OPSD-Team-email,
      web: http://open-power-system-data.org/

views:
    # You can put hints here which kind of graphs or maps make sense to display your data. This makes the 
    # Data Package Viewer at http://data.okfn.org/tools/view automatically display visualazations of your data.
    # See http://data.okfn.org/doc/data-package#views for more details.    

# extend your datapackage.json with attributes that are not
# part of the data package spec
# you can add your own attributes to a datapackage.json, too

openpowersystemdata-enable-listing: True  # This is just an example we don't actually make use of yet.


"""

metadata = yaml.load(metadata)

datapackage_json = json.dumps(metadata, indent=4, separators=(',', ': '))

# Write results to file

In [ ]:
output_path = 'output/datapackage_generation_capacities/'
output_path2 = 'output/datapackage_generation_capacities'

#Write the result to file
data_raw.to_csv(output_path+'national_generation_capacities_crosstab.csv', encoding='utf-8')

#Write the results to excel file
data_raw.to_excel(output_path+'national_generation_capacities_crosstab.xlsx', sheet_name='output')

#Write the result to file
data.to_csv(output_path+'national_generation_capacities.csv', encoding='utf-8')

#Write the results to excel file
data.to_excel(output_path+'national_generation_capacities.xlsx', sheet_name='output')

#Write the information of the metadata
#with open(os.path.join(output_path, 'datapackage.json'), 'w') as f:
#    f.write(datapackage_json)

#Set this string to this notebook's filename!    
#nb_filename = 'National_Generation_Capacities.ipynb'

# Save a copy of the notebook to markdown, to serve as the package README file
#subprocess.call(['ipython', 'nbconvert', '--to', 'markdown', nb_filename])
#path_readme = os.path.join(output_path2, 'README.md')
#try:
#    os.remove(path_readme)
#except Exception:
#    pass
#os.rename(nb_filename.replace('.ipynb', '.md'), path_readme)    
 